In [77]:
import pandas as pd
import numpy as np

The postal code data was copied in an excel sheet. Installing the xlrd package in the cell below as it is required to read excel sheet. 

In [ ]:
pip install xlrd

In [78]:
import xlrd

In [81]:
df = pd.read_excel('CanadaPostalCodes.xlsx')

Dropping the rows from the dataframe where the Bororugh column is equal to 'Not assigned'

In [82]:
df = df.drop(df[df['Borough'] == 'Not assigned'].index)

Setting the Neighborhood of the rows that are 'Not assigned' to the corresponding value of the 'Borough' column

In [83]:
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.loc[df.Neighborhood == 'Not assigned', 'Borough']

Merging the rows that have the same values for the 'PostalCode' and the 'Borough' columns

In [85]:
df = df.groupby(["PostalCode", "Borough"]).agg( ','.join)

In [88]:
df.shape

(103, 1)

Copied the data from the csv at the URL at https://cocl.us/Geospatial_data in an Excel sheet and renamed the column 'Postal code' to 'PostalCode' same as in the excel sheet CanadaPostalCodes.xlsx. Loaded the data in a dataframe called geodf

In [107]:
geodf = pd.read_excel('geocoder.xlsx')

Merging the data in the dataframe 'geodf' into the dataframe 'df' based on the column 'PostalCode'

In [105]:
df = df.merge(geodf, on="PostalCode", how = 'inner')